# **Functional 모델: 다중출력**

<center><img src = "https://github.com/Jangrae/img/blob/master/carseats2.png?raw=true" width=800/></center>

## **1. 환경준비**

### (1) 라이브러리 불러오기

In [ ]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam

%config InlineBackend.figure_format='retina'

### (2) 함수 만들기

In [ ]:
# 함수 만들기
def dl_history_plot(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history['loss'], label='Train Loss', marker='.')
    plt.plot(history['val_loss'], label='Validation Loss', marker='.')

    plt.title('Learning Curve', size=15, pad=20)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()

### (3) 데이터 준비

In [ ]:
# 데이터 준비
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/Carseats.csv'
data = pd.read_csv(path)
data.drop(columns='Education', inplace=True)
data.head()

**데이터 설명**

- Sales: 각 지역 판매량(단위: 1,000개)
- CompPrice: 경쟁사 가격(단위: 달러)
- Income: 지역 평균 소득(단위: 1,000달러)
- Advertising: 각 지역, 회사의 광고 예산(단위: 1,000달러)
- Population: 지역 인구수(단위: 1,000명)
- Price: 자사 지역별 판매가격
- ShelveLoc: 진열상태
- Age: 지역 인구의 평균 연령
- Urban: 도심 지역 여부(Yes, No)
- US: 매장이 미국에 있는지 여부(Yes, No)

## **2. 데이터 전처리**

### (1) 데이터 준비

In [ ]:
# x, y 분리
target1 = 'Sales'
target2 = 'US'
data[target2] = np.where(data[target2] == 'Yes', 1, 0)

### (2) 가변수화

In [ ]:
# 가변수화
cat_cols = ['ShelveLoc',  'Urban']
data = pd.get_dummies(data, columns=cat_cols, drop_first=True, dtype=int)

### (3) 데이터 분할

In [ ]:
# 학습용, 검증용 분리
train, val = train_test_split(data, test_size=0.2, random_state=1)

In [ ]:
# 출력 나누기
x_train = train.drop(columns=[target1, target2])
y_train1 = train.loc[:, target1]
y_train2 = train.loc[:, target2]

x_val = val.drop(columns=[target1, target2])
y_val1 = val.loc[:, target1]
y_val2 = val.loc[:, target2]

### (4) 스케일링

In [ ]:
# 스케일링
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

## **3. 모델링**

### (1) 모델 선언

In [ ]:
# 메모리 정리
clear_session()

# 입력 Feature 수
n_feature = x_train.shape[1]

# 입력
il = Input(shape=(n_feature, ))

# Hidden Layer 선언
h1 = Dense(32, activation='relu')(il)
h2 = Dense(16, activation='relu')(h1)

# 첫 번째 Output Layer 선언
h3_1 = Dense(4, activation='relu')(h2)
o1 = Dense(1)(h3_1)

# 두 번째 Output Layer 선언
h3_2 = Dense(4, activation='relu')(h2)
o2 = Dense(1, activation='sigmoid')(h3_2)

# 모델 선언
model = Model(inputs=il, outputs=[o1, o2])

# 모델 요약
model.summary()

In [ ]:
# 모델 구조 시각화
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot

dot = model_to_dot(model, show_shapes=True, rankdir='TD')
dot.set_graph_defaults(dpi='60')
SVG(dot.create(prog='dot', format='svg'))

### (2) 모델 학습

In [ ]:
# 학습 설정
model.compile(optimizer=Adam(learning_rate=0.01), loss=['mse', 'binary_crossentropy'])

In [ ]:
# 모델 학습
hist = model.fit(x_train, [y_train1, y_train2], epochs=50, validation_split=0.2, verbose=0).history

In [ ]:
# 학습 곡선
dl_history_plot(hist)

### (3) 예측 및 성능 평가

In [ ]:
# 예측
y_pred = model.predict(x_val)

In [ ]:
# 성능 평가 #1
print('Sales Prediction:')
print('* MAE:', mean_absolute_error(y_val1, y_pred[0]))
print('* R2:', r2_score(y_val1, y_pred[0]))

In [ ]:
# 성능 평가 #2
y_pred2 = np.where(y_pred[1] > 0.5, 1, 0)
print('US Prediction:')
print(confusion_matrix(y_val2, y_pred2))
print(classification_report(y_val2, y_pred2))